# Power Spectrum 
## Calculate the power spectrum using a 3D grid
Generate a the density field of the galaxy positions using a 3D grid
Using the 3D grid, then calculate the power spectrum using the powerbox package

In [ ]:
# Power Spectrum plot
import matplotlib.pyplot as plt
import numpy as np
import powerbox as pbox
from Functions import *
from mpl_toolkits import mplot3d
from powerbox import get_power
from scipy import stats
import statistics
import pandas as pd

# Redshift = 0 
#get gals, snaps, sims_props
snapshot_used = 250 # z = 0
gals, sim_props, snaplist = get_gal_catalogue(snapshot_used)
#ngals = len(gals)
#print(ngals)
#gals.columns #returns all avaiable properties of galxies
#gals[gals.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]# remove infs
#gals.dropna()# drop nan values

# See other available Snaps and Redshifts
reds = snaplist[1]
snaps = snaplist[0]
print(reds)
print(snaps)

# using positions [x,y,x]
pos_x = gals["Pos_0"]
pos_y = gals["Pos_1"]
pos_z = gals["Pos_2"]

Pos = np.array([pos_x, pos_y, pos_z])
Pos.T.shape
Pos_new=Pos.T # Transpose the Pos so it is in format (N,D) array
Pos_new.shape

snaplist_index_used = list(snaplist[0]).index(snapshot_used)# convert to list and find the index of the snap we are using
redshift_used = snaplist[1][snaplist_index_used] # Get the redshift for corresponding snapshot
redshift_used = str(round(redshift_used, 1)) # round float to nearest whole int
gals.columns #returns all avaiable properties of galxies

#sim_props # returns sim props if you want to check

In [ ]:
gals["HIMass"]


In [ ]:
HIMass = gals["HIMass"]

# get mass and convert to proper units
HIMass = np.log10(gals["HIMass"]*1e10)
HIMass, min(HIMass), max(HIMass)
# Get rid of inf's in HIMass
HIMass=np.nan_to_num(HIMass)
HIMass=HIMass[np.nonzero(HIMass)]

# MAKE 3D array for grid later
#HIMass = np.array([HIMass, HIMass, HIMass])
HIMass = np.array(HIMass, 3)
HIMass.T.shape
HIMass_new=HIMass.T # Transpose the HIMass so it is in format (N,D) array
#HIMass_new.shape
HIMass = np.array([HIMass, 3])
HIMass.T.shape
HIMass_new=HIMass.T
HIMass_new

## Create 3D grid of galaxy positions
### 1. Calculate the edges of the grid
### 2. Calculate the deltax
    delta x = (Pos/edges)
### 3. Get the 3D grid
    using histogram 3D

In [ ]:
# getting bits ready for 3D grid 
boxsize = 178.57142857142858 # get the box size, this is from sims pops 
dim = len(Pos) # number of dimensions in 3D space =3
boxlength = [boxsize] * dim #get box length for each side of box
N = pos_x.shape # = 1648367
#N = 100
V = np.product(boxlength)
#print('dim : ', dim)
#print('boxlength : ', boxlength)
#print('N : ', N)
#print('V : ', V)
#N_dim = [N] * dim # to make iterable
#print('N : ', N)
#print('Pos : ', Pos)
#Pos = np.squeeze(Pos)
#print('Pos : ', Pos)
#N[0] / boxsize

# Create 3D grid for 
## 1. Galaxy positions ($\rho_{x}$)

In [ ]:
nbins=100 # define the number of bins for our 3D grid
rhox, edges = np.histogramdd(Pos_new, bins=nbins) #[0].astype("float") # calculate the 3D grid using histogramdd
rhox.shape, edges[0].size, edges[1].size, edges[2].size
#so its done 10 x omega = deltax / np.mean(deltax) - 1 # calcualte the over density
#print('omega: ', omega)10 x 10 by default
rhox # This returns the number of galaxies in each grid

## 2. HIMass $\rho_{HIMass}$

In [ ]:
#rhoHIMass, edgesHIMass = np.histogramdd(HIMass_new, bins=nbins, normed=True) #[0].astype("float") # calculate the 3D grid using histogramdd
#rhoHIMass.shape, edgesHIMass[0].size # 1D only

rhoHIMass, edgesHIMass = np.histogramdd(Pos_new, bins=nbins) #[0].astype("float") # calculate the 3D grid using histogramdd
rhoHIMass.shape, edgesHIMass[0].size # 1D only
rhoHIMass

## Calculate the density using the mean of the density field:

$\delta = \frac{\rho(x)}{\bar{\rho(x)}}$

### Galaxies:

In [ ]:
delta = rhox / np.mean(rhox) - 1 # calcualte the over density
#print('omega: ', omega)
delta

## HI Mass 

In [ ]:
delta_HIMass = rhoHIMass / np.mean(rhoHIMass) - 1 # calculate the over density
#print('omega: ', omega)
delta_HIMass

## Now lets do the Galaxy and HI Mass Power Spectrum 
### The only difference is that we add weights (HI_Mass is the weights for the PS)

In [ ]:
# REDSHIFT z=0
# GALAXY PS = gal_ps, gal_k
# HIMASS PS = HIMass_ps, HIMass_k
gal_ps, gal_k = get_power(delta,boxlength,remove_shotnoise=False,weights=None)# NO WEIGHTS for GALAXY PS
HIMass_ps, HIMass_k = pbox.get_power(delta,boxlength,remove_shotnoise=False,weights=rhoHIMass)# WEIGHTS = HIMASS for HIMASS PS

# test first, then try less parameters if it goes wierd : p_k_field, bins_field = get_power(omega, boxlength)

#print('gal_ps (Galaxies): ', gal_ps)
#print('gal_k (Galaxies): ', gal_k)
#print('HIMass_ps (HI Mass): ', HIMass_ps)
#print('HIMass_k (HI Mass): ', HIMass_k)

# PLOT
## GALAXY PS

In [ ]:
fig, ax = plt.subplots(1, 1)
label_z = ("z = " + redshift_used)
#ax.set_ylim([0,3500])
#ax.set_xlim([0.1,1])
label_galsPS = ("Galaxy Power Spectrum")
label_HIMassPS = ("HI Mass Power Spectrum")
ax.set_xlabel(r"$k$ $[Mpc^{-1}]$")
ax.set_ylabel(r"$P_{HI}(k)$ $[Mpc^{3}]$")
#plt.xscale('log')
#plt.yscale('log')
ax.plot(gal_k,gal_ps,color="black", label=label_galsPS)
ax.plot(HIMass_k,HIMass_ps,color="deepskyblue", label=label_HIMassPS)

plt.legend(title=("DRAGONS"),
           loc='upper right',
           fontsize=12)

nbins = str(nbins)
name_of_plot=("PowerSpectrum_GALS_and_HIMass_unlogged" + nbins)
plt.savefig("plots/" + name_of_plot + ".pdf")

# HI MASS PS

fig, ax = plt.subplots(1, 1)
label_z = ("z = " + redshift_used)
#ax.set_ylim([0,3500])
#ax.set_xlim([0.1,1])
ax.set_xlabel(r"$k$ $[Mpc^{-1}]$")
ax.set_ylabel(r"$P_{HI}(k)$ $[Mpc^{3}]$")
plt.xscale('log')
plt.yscale('log')
ax.plot(bins_field_HI_Mass,p_k_field_HI_Mass,color="deepskyblue", label=label_z)

plt.legend(title=("DRAGONS"),
           loc='upper right',
           fontsize=12)

nbins = str(nbins)
name_of_plot=("PowerSpectrum_HIMass" + nbins)
plt.savefig("plots/" + name_of_plot + ".pdf")